# Ayudantía 02

## Programación Funcional

Vicente Águila y Paul Heinsohn

**Form de Feedback:** https://docs.google.com/forms/d/1rublnCunwYWYe2QxARiND1hS9WJ9jKLE6WJilA7rRvE

## ¿Por qué?

* Existen lenguajes de programación que utilizan este paradigma (Erlang)
* Procesamiento de información optimizada

________________________________________________________________________________

## Introducción
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Llevas todo el día estudiando la materia de Programación funcional que ya no das más, por lo que decides ver <strike>muchas</strike> una película, sin embargo, hay tantas películas buenas como ’Buscando a Enzo’, ’Las 50 Sombras de Hernan’, ’Lo Que Benja Se Llevó', etc. que no logras decidirte. Luego de un rato, llegas a la brillante idea de crear un programa que te ayude a resolverlo, para esto ingresas a Prograflix a ver como puedes comenzar.

<img src="img/prograflix.png">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;El objetivo es realizar un sistema de consultas que te permita obtener información sobre películas y actores. Para esto, obtienes mágicamente la base de datos de <i>Prograflix</i> organizada en dos archivos especificados a continuación:
    
<ul>
    <li><b>peliculas.txt:</b> Archivo de texto que contiene la información de las películas que puedes ver, de la forma: `id, nombre, puntuacion, fecha_estreno, genero`.</li><br>
    <li><b>actores.txt:</b> Archivo de texto que alberga la información de los artistas que trabajaron en las películas. Está distribuido de la forma: `nombre_pelicula, nombre_actor, nombre_personaje`.</li>
</ul>

Con esta información debes ser capaz de resolver las consultas especificadas siempre y cuando se cumpla lo siguiente:

<ul>
    <li>Tu solución debe ser realizada con <b>programación funcional</b></li><br>
    <li>Quieres que tu programa no almacene el archivo en memoria, ergo, la lectura tendrá que ser sólamente mediante el uso de generadores</li><br>
    <li>Cada película debe tener su propio id que debe estar implementado con <b>generadores</b>. El id es distinto al que trae la base de datos, para que puedas distinguirla con tu programa.</li><br>
</ul>

Luego, lo primero que debemos hacer es crear el id para las películas:
</div>

In [16]:
def generador_id():
    n=0
    while True:
        yield n
        n+=1

Procedemos a definir las películas y los actores:

In [41]:
from collections import namedtuple

Pelicula = namedtuple("Pelicula",["id", "nombre", "puntuacion", "fecha_estreno", "genero"])
Actores = namedtuple("Actor",["nombre_pelicula","nombre_actor","personaje"])

<div style="text-align: justify">Ahora faltaría abrir los archivos para instanciar los actores y las películas. Hasta ahora solo hemos trabajado los archivos guardándolos en memoria mediante "for's" tediosos... pero qué tal si lo hacemos con listas por compresión:
</div>

In [42]:
limpeador = lambda linea: linea.strip().split(",")
gen_id = generador_id()
with open("archivos/peliculas.txt", "r" , encoding = "utf8") as archivo:
    PELICULAS = [Pelicula(next(gen_id), *limpeador(linea)[1:]) for linea in archivo]

with open("archivos/actores.txt", "r" , encoding = "utf8") as archivo:
    ACTORES = [Actores(*limpeador(linea))for linea in archivo]

________________________________________________________________________________
Dentro de las consultas que nuestro programa debe soportar veremos los siguientes:

* Crear el método <b>popular</b> que dado un número <b>n</b>, retorne todas las películas que tienen un rating superior a dicho valor.

In [43]:
def popular(peli, n):
    return filter(lambda p: float(p.puntuacion) >= (n), peli)

In [44]:
print([top.nombre for top in popular(PELICULAS, 20)])

['Fantastic Beasts and Where to Find Them', 'Doctor Strange', 'Kong: Skull Island', 'Arrival', 'Guardians of the Galaxy', 'Rings', 'Split', 'John Wick', 'Mad Max: Fury Road', 'Jurassic World', 'Logan', 'Fifty Shades Darker', 'Beauty and the Beast', 'Captain America: Civil War', 'Interstellar', "Assassin's Creed"]


________________________________________________________________________________
* Cree la función **actores_genero** que retorne el **nombre** de todos los actores que actuaron en una película del **género** especificado. 

In [66]:
def actores_genero(gen, l_actores, l_peliculas):
    lista_generos_peli = filter(lambda p: p.genero == gen, l_peliculas)
    nombre_genero = list(map(lambda a: a.nombre, lista_generos_peli))
    
    actores_peli = filter(lambda a: a.nombre_pelicula in nombre_genero, l_actores)
    return map(lambda a: a.nombre_actor, actores_peli)
actores_genero("Comedy", ACTORES, PELICULAS)


In [67]:
print(*list(actores_genero("Comedy", ACTORES, PELICULAS))[:10], sep="\n")

Kaley Cuoco
Zoey Deutch
Jessica Barth
Stephen Root
Blake Jenner
Jennifer Aniston
Courtney B. Vance
Seth MacFarlane
Megan Mullally
Lewis Black


________________________________________________________________________________
* Crear el método <b>puntuacion_actor</b> que dado el <b>nombre de 'Hugh Jackman'</b>, retorna el promedio de la puntuación de las películas en las que ha participado.

In [95]:
from functools import reduce

def puntuacion_actor(l_peli, l_actores, n_actor):
    peli_actua = filter(lambda a: a.nombre_actor == n_actor, l_actores)
    nombre_plei = list(map(lambda a: a.nombre_pelicula, peli_actua))
    
    t_peli = filter(lambda p: p.nombre in nombre_plei, l_peli )
    l_punt = list(map(lambda p: p.puntuacion, t_peli))
    return float(reduce(lambda x, y: float(x) + float(y), l_punt)) / len(l_punt)
    
    
puntuacion_actor(PELICULAS, ACTORES, "Hugh Jackman")

58.03259566666666

In [96]:
print("Hugh Jackman's rating: {}".format(puntuacion_actor(PELICULAS, ACTORES, "Hugh Jackman")))

Hugh Jackman's rating: 58.03259566666666


* Crear el método **puntuaciones_actores** que retorne el promedio de la puntuación de las películas en que cada actor ejerce su trabajo.

In [105]:
def puntuaciones_actores(l_peli, l_actores):
    list_nombre = list(map(lambda x: x.nombre_actor, l_actores))
    list_puntuacion = list(map(lambda x: puntuacion_actor(l_peli, l_actores, x),list_nombre))
    return list(zip (list_nombre,list_puntuacion))
    
    

In [106]:
puntuaciones_actores(PELICULAS, ACTORES)

[('Dafne Keen', 157.275668),
 ('Sandra Bullock', 11.594164),
 ('Allison Janney', 11.07952225),
 ('Brad Garrett', 7.642378),
 ('Allison Janney', 11.07952225),
 ('Mark Wahlberg', 7.2816855),
 ('Hugh Jackman', 58.03259566666666),
 ('Kaley Cuoco', 16.928488),
 ('Aldis Hodge', 8.192746),
 ('Jason Clarke', 10.2978015),
 ('Kevin Spacey', 5.697074),
 ('Symara A. Templeman', 5.887713),
 ('Natalie Portman', 7.4017905),
 ('Michele Valley', 5.872569),
 ('Terrence Howard', 7.292687),
 ('Cillian Murphy', 7.812282999999999),
 ('Li Bingbing', 6.253452),
 ('Tom Hanks', 6.619415500000001),
 ('Kavita Patil', 9.408097),
 ('Michael Caine', 11.8650926),
 ('Jeremy Irons', 15.5576675),
 ('Dave Franco', 6.742778),
 ('Keira Knightley', 9.031057800000001),
 ('Michael Fassbender', 14.187058333333335),
 ('Bill Nighy', 7.919401666666666),
 ('Shakira', 10.603307),
 ('Jordan Nagai', 7.456567),
 ('Valeria Cavalli', 19.591843),
 ('Analeigh Tipton', 7.817199),
 ('Mahershala Ali', 9.346346),
 ('Miles Christopher Bakshi',